In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, Latex
df = pd.read_csv('responses_scrubbed.csv', index_col=None, parse_dates=['date'])

#don't truncate results
pd.set_option('display.max_rows', None)

## Number of respondents from each neighborhood
Gives us a sense of who responded.

In [2]:
df['neighborhood'].value_counts()

Harmon                 158
Upper Village          111
Old Post Road North     71
Mount Airy              60
Sunset Park             46
Albany Post Road        29
Half Moon Bay           19
Quaker Ridge            10
Cortlandt                9
Wells/Beekman Area       1
Ossining                 1
Name: neighborhood, dtype: int64

## Number of respondents from each demographic
Gives us a sense of who responded.

In [3]:
df['demographic'].value_counts()

Adult with school kid                                          210
Adult without school kid                                       168
Senior                                                         134
College kid                                                      5
High School kid                                                  3
Middle School kid                                                1
Adult with school age children and a senior living with me.      1
Name: demographic, dtype: int64

## Percent respondents of each demographic from each neighborhood
Each column adds up to 100%.

In [4]:
df2 = df.groupby(['demographic', 'neighborhood']).size().unstack(fill_value=0).apply(lambda x: 100 * x / x.sum()) #.astype(int)
df2.round(0).astype(int) # round to int

neighborhood,Albany Post Road,Cortlandt,Half Moon Bay,Harmon,Mount Airy,Old Post Road North,Ossining,Quaker Ridge,Sunset Park,Upper Village,Wells/Beekman Area
demographic,,,,,,,,,,,
Adult with school age children and a senior living with me.,0,0,0,0,0,0,0,0,0,1,0
Adult with school kid,21,33,5,40,50,46,0,50,46,42,100
Adult without school kid,28,44,47,27,27,34,100,30,39,34,0
College kid,0,0,0,2,0,1,0,0,2,0,0
High School kid,0,0,0,1,2,0,0,0,0,0,0
Middle School kid,0,0,0,1,0,0,0,0,0,0,0
Senior,52,22,47,30,22,19,0,20,13,24,0


## Analyze questions with only one answer option allowed


In [5]:
# a function that calculates how many responses were given in a specific column
def get_num_respondents(field, df):
    # get the number of responses in this field
    num_respondents = df[pd.notnull(df[field])].shape[0]
    return num_respondents

def get_breakdown(field, breakdown_field, percent=True):
    # get the number of respondents to this field
    num_respondents = get_num_respondents(field, df)

    # get the overall stats of the values in the field of interest
    df2 = df[field].value_counts()
    if percent:
        df2 = df2.apply(lambda x: int(100 * x / num_respondents))
    
    # label the column nicely 
    df2 = df2.to_frame()
    df2.columns = ['overall'] #name this 'overall', since it is not broken down

    # get the various values given in the field we want to break down the target field by
    breakdowns = df[breakdown_field].value_counts().items()

    # now break down the field by each of the values in the breakdown field
    for breakdown, value in breakdowns:        
        # get the number of responses by respondents matching this breakdown category
        df3 = df[df[breakdown_field] == breakdown][field].value_counts()
        
        # convert to percentage, if desired
        if percent:
            # get the number of respondents to this field
            num_respondents_column = get_num_respondents(field, df[df[breakdown_field] == breakdown][field].to_frame())
            
            df3 = df3.apply(lambda x: int(100 * x / num_respondents_column))

        # label the column nicely before we merge it into the other dataframe
        df3 = df3.to_frame() # convert to dataframe
        df3.columns = [breakdown]
        

        # join these two dataframes together
        df2 = df2.join(df3, how='outer')

    return df2

# fields of interest
fields = [ 'child_bus_freq', 'child_walk_freq', 'child_driven_freq', 'child_drive_freq', 'child_bike_freq', 'walk_freq', 'bike_freq', 'bikes_on_sidewalk', 'self_jog_frequency', 'commutes', 'child_self_school', 'child_self_bus_freq', 'child_self_bike_freq', 'child_self_driven_freq', 'child_self_drive_freq', 'child_self_walk_freq', 'child_self_commutes', 'child_self_has_children', 'commuter_type', 'commuter_walk_to_station_freq', 'commuter_bike_to_station_freq', 'commuter_drive_to_station_freq', 'commuter_carpool_to_station_freq', 'commuter_driven_to_station_freq', 'commuter_bus_to_station_freq', 'drivers_are_safe', 'bicyclists_are_safe', 'contact_interest' ]
# not included: ['child_drive_reason', 'child_no_walk_reason', 'child_no_bike_reason', 'commuter_distance', 'no_walk_reason', 'no_bike_reason', 'drive_reason' ]

# iterate through every field of interest
for field in fields:
    
    # get this field's breakdown by demographic and neighborhood
    df2 = get_breakdown(field, 'demographic', percent=True)
    df3 = get_breakdown(field, 'neighborhood', percent=True)
    
    # print out results
    display(Markdown('## *{}* (each column adds up to 100%):'.format(field, get_num_respondents(field, df)).title()))

    display(Markdown('### by demographic:'.format(field, get_num_respondents(field, df)).title()))
    display(df2)

    display(Markdown('### by neighborhood:'.format(field, get_num_respondents(field, df)).title()))
    display(df3)


## *Child_Bus_Freq* (Each Column Adds Up To 100%):

### By Demographic:

,overall,Adult with school kid,Adult without school kid,Senior,College kid,High School kid,Middle School kid,Adult with school age children and a senior living with me.
"Every day, or almost every day",49,49,NaN,NaN,NaN,NaN,NaN,NaN
Rarely or never,40,40,NaN,NaN,NaN,NaN,NaN,NaN
Once in a while,5,5,NaN,NaN,NaN,NaN,NaN,NaN
Once every few days,3,3,NaN,NaN,NaN,NaN,NaN,NaN


### By Neighborhood:

,overall,Harmon,Upper Village,Old Post Road North,Mount Airy,Sunset Park,Albany Post Road,Half Moon Bay,Quaker Ridge,Cortlandt,Wells/Beekman Area,Ossining
"Every day, or almost every day",49,39,48.0,56,70,42.0,50.0,100.0,20.0,66.0,100.0,NaN
Once every few days,3,6,NaN,6,3,4.0,NaN,NaN,NaN,NaN,NaN,NaN
Once in a while,5,9,6.0,3,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rarely or never,40,44,44.0,34,20,52.0,50.0,NaN,80.0,33.0,NaN,NaN


...

## *Contact_Interest* (Each Column Adds Up To 100%):

### By Demographic:

,overall,Adult with school kid,Adult without school kid,Senior,College kid,High School kid,Middle School kid,Adult with school age children and a senior living with me.
No,53,51,53,55,60,100.0,100.0,NaN
Yes,46,48,46,44,40,NaN,NaN,100.0


### By Neighborhood:

,overall,Harmon,Upper Village,Old Post Road North,Mount Airy,Sunset Park,Albany Post Road,Half Moon Bay,Quaker Ridge,Cortlandt,Wells/Beekman Area,Ossining
No,53,51,51,55,58,52,59,66,40,55,NaN,100.0
Yes,46,48,48,44,41,47,40,33,60,44,100.0,NaN


## Who rides on the sidewalk?

### Overall

In [6]:
# do people generally bike on the sidewalk?
df['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bikes_on_sidewalk
No - I don't bicycle,48
No - I only bicycle on the road,29
Yes,23


### People who never bike

In [7]:
# do people who never bike claim to ride on the sidewalk?
df2 = df[ df['bike_freq'].isin(['Rarely or never']) ]
df2['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bikes_on_sidewalk
No - I don't bicycle,81
No - I only bicycle on the road,13
Yes,7


### People who bike at least occasionally

In [8]:
# do people who ride a bike occasionally or more ride on the sidewalk?
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day', 'Once every few days', 'Once in a while']) ]
df2['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bikes_on_sidewalk
No - I don't bicycle,1
No - I only bicycle on the road,53
Yes,46


### People who bike a lot

In [9]:
# do people who bike a lot ride on the sidewalk?
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day']) ]
df2['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bikes_on_sidewalk
No - I only bicycle on the road,61
Yes,39


## Who thinks drivers are safe?

### Overall

In [10]:
# do people generally think drivers are safe?
df['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()

,drivers_are_safe
1.0,9
2.0,30
3.0,47
4.0,14


### People who walk a lot

In [11]:
# do people who walk a lot think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,drivers_are_safe
1.0,10
2.0,33
3.0,46
4.0,11


### People who rarely walk

In [12]:
# do people who rarely walk think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,drivers_are_safe
1.0,6
2.0,24
3.0,48
4.0,22


### People who never walk

In [13]:
# do people who never walk think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Rarely or never']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,drivers_are_safe
1.0,10
2.0,22
3.0,41
4.0,27


### People who bike frequently

In [14]:
# do people who bike a lot think drivers are safe?
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()

,drivers_are_safe
1.0,12
2.0,32
3.0,48
4.0,7


## Who thinks bicyclists are safe?

### Overall

In [15]:
# do people generally think bicyclists are safe?
df['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bicyclists_are_safe
1.0,13
2.0,23
3.0,49
4.0,15


### People who bike a lot

In [16]:
# do people who bike a lot think bicyclists are safe
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bicyclists_are_safe
1.0,4
2.0,15
3.0,56
4.0,25


### People who rarely bike

In [17]:
# do people who never bike think bicyclists are safe
df2 = df[ df['bike_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bicyclists_are_safe
1.0,15
2.0,26
3.0,46
4.0,13


In [18]:
# do people who never bike think drivers are safe?
df2 = df[ df['bike_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,drivers_are_safe
1.0,8
2.0,29
3.0,47
4.0,16


In [19]:
df[df['demographic'] == 'College kid']['walk_freq'].value_counts().sort_index()

Once every few days    2
Once in a while        2
Rarely or never        1
Name: walk_freq, dtype: int64

# Analyze responses to questions that allow more than one answer to be given

In [20]:
# collects and displays how many respondents selected each answer option for questions that allowed more than one answer
def display_answer_counts(field_prefixes):
    # loop through each of these question prefixes so we can find the corresponding columns
    for prefix in field_prefixes:
        # get all the column names for each question
        column_names = [c for c in df if c.startswith(prefix + "_")]
        #display(column_names)

        #merge all the columns into one so we can do some simple overall stats on them
        df2 = pd.melt(df, id_vars=['neighborhood', 'demographic'], value_vars=column_names, value_name=prefix)

        #count how many times each response was given, as a percent of all
        counts = df2[prefix].value_counts().to_frame().apply(lambda x: 100 * x / get_num_respondents(prefix, df2)).round(0).astype(int) 

        display(Markdown('## {} (as % of respondents to this question)'.format(prefix, get_num_respondents(prefix, df2)).title()))
        display(counts)

# a list of dataframe column prefixes for questions that allowed multiple answers given
fields = [ 'schools', 'child_drive_reason', 'child_no_walk_reason', 'child_no_bike_reason', 'commuter_distance', 'no_walk_reason', 'no_bike_reason', 'drive_reason' ]
display_answer_counts(fields)


## Schools (As % Of Respondents To This Question)

,schools
CET,36
PVC,25
CHHS,23
Homeschooled,2
Circle School,1
College,0
Windward and Hackley,0
St. Augustine,0
St. Augustine,0
Happy Hearts,0


## Child_Drive_Reason (As % Of Respondents To This Question)

,child_drive_reason
No - we do not drive,22
or prefer not to drive,22
Our own personal preference,14
The bus schedule does not match our schedule,9
Safety concerns with walking,8
Safety concerns with bicycling,5
Lack of available busing where we live,2
Safety concerns with buses,2
Kids running late,1
Weather,1


## Child_No_Walk_Reason (As % Of Respondents To This Question)

...

,no_walk_reason
No - I walk a lot,28
Lack of adequate sidewalks,15
I don't have time to walk,11
My own personal preference,9
Fear of dangerous driving,8
Lack of adequate crosswalks at intersections,6
I live too far to walk,6
My health condition,4
Visually unappealing,3
I do not like to walk,1


## No_Bike_Reason (As % Of Respondents To This Question)

,no_bike_reason
Fear of dangerous driving,18
Lack of adequate bike lanes,18
My own personal preference,13
I do not like to bicycle,12
I don't have time to bicycle,9
No - I bicycle a lot,8
My health condition,3
I live too far to bicycle,2
Visually unappealing,1
Hills,1


## Drive_Reason (As % Of Respondents To This Question)

,drive_reason
My own personal preference,25
I don't have time to walk or bicycle,19
Safety concerns with bicycling,13
Safety concerns with walking,8
or prefer not to drive,5
No - I do not drive,5
My own health condition,3
Hills,1
Convenience,0
Time,0


## Answers to questions with free text responses

In [21]:
# a list of dataframe column prefixes for questions that allowed free text responses
fields = ['feelings', 'problem_areas', 'suggested_improvements', 'additional_comments', 'business_additional_comments', 'final_comments']

# setting so that it displays the full comment
pd.set_option('display.max_colwidth', -1)

# show all responses to each question
for field in fields:
    display(Markdown('## {} (from {} respondents)'.format(field, get_num_respondents(field, df)).title()))
    df2 = df[pd.notnull(df[field])]
    display(df2[field])



## Feelings (From 394 Respondents)

0      Okay, pretty good                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## Problem_Areas (From 384 Respondents)

0      Croton Point Avenue, shoprite, riverside                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

## Suggested_Improvements (From 386 Respondents)

...

1      I think much of the public supports slowing down traffic.  The hard job is gaining the political will to take bold steps that will inevitably attract some public backlash.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

## Business_Additional_Comments (From 12 Respondents)

22     none                                                                                                                                                                                                                                                    
64     My students are terrified of backing out my driveway due to the speeding cars.  A mother with her two eight year old twins and her twelve year old daughter came so close to being broadsided by a car speeding down Mt Airy, they were lucky to escape.
78     Enforce speed limits in general...thx!                                                                                                                                                                                                                  
143    because our village has 3 distinct business districts, it’s a difficulty task connecting them for walking                                                                                                                        

## Final_Comments (From 87 Respondents)

8      Thank you!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
13     Suburban US is a very car-centric culture. it's hard to change this but we can try!                                                                                                                                                         